In [ ]:
#Following tutorial found here: https://sep.readthedocs.io/en/v1.0.x/tutorial.html
#importing matplotlib, numbpy, astropy (to read fits file), and sep (a source detection package)



%matplotlib inline
from astropy.io import fits
import numpy as np
import sep
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = [10., 8.]

In [ ]:
#opening the sample image from the tutorial

fname = "image.fits"
hdu_list = fits.open(fname)
hdu_list.info()

In [ ]:
#reading and copying image information to im_data (2d numpy array).


im_data = hdu_list[0].data
hdu_list.close()


In [ ]:
#displaying the image and saving as a png file


m, s = np.mean(im_data), np.std(im_data)
plt.imshow(im_data, interpolation='nearest',cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()

plt.savefig("image.png")

In [ ]:
#measuing the background of the image using sep.Background()


bkg = sep.Background(im_data)

#getting global (overall) mean and noise of image background
print (bkg.globalback)
print (bkg.globalrms)

In [ ]:
#detailing background image as a 2d array as was done with the original image


bkg_image = bkg.back()


#showing the image of the background, and saving as a png file
plt.imshow(bkg_image, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar()

plt.savefig("bkg_image.png")

In [ ]:
#Detailing the background noise as a 2-D array

bkg_rms = bkg.rms()

#Showing the background noise image, and saving as a png file

plt.imshow(bkg_rms, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar()

plt.savefig("bkg_rms.png")

In [ ]:
#subtracting the background from the original image 
data_sub = im_data - bkg

In [ ]:
#using sep to extract objects in the image after background has been subtracted. Then finding total amount of objects

objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
len(objects)

In [ ]:
#To detect where the aforementioned sources are, they will be outlined in a red ellipse. The subsequent image is then shown
#and the file is saved as a png file
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation = 'nearest', cmap = 'gray', vmin = m-s, vmax = m+s, origin = 'lower')

for i in range(len(objects)):
    e = Ellipse(xy =(objects['x'][i], objects['y'][i]), 
               width = 6*objects['a'][i],
               height = 6*objects['b'][i],
               angle=objects['theta'][i] * 180./np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

plt.savefig("sub_bkg.png")

In [ ]:
#Reviewing the many attributes of objects (the different properties of the sources, such as flux, or theta)

objects.dtype.names


In [ ]:
#Performing photometry (finding flux) for 10 objects. 

flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 
                                    3.0, err=bkg.globalrms, gain=1.0)

for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))